In [2]:
# Library loads
import numpy as np
import sklearn
import pandas as pd
import os

from numpy import random
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn import svm

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import Normalizer
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score, mean_squared_error,mean_absolute_error, roc_curve, classification_report, auc)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# For creating plots using data 
'''
import seaborn as sns
import plotly
from plotly import __version__
print(plotly.__version__)
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()
import plotly.io as pio
pio.renderers.default = 'colab'
'''
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
from numpy.random import seed
seed(1)

seed=1
pd.options.display.float_format = '{:, .0f}'.format
data_2014 = pd.read_csv('/content/drive/My Drive/Data/2014_Financial_Data.csv', header=0)
data_2015 = pd.read_csv('/content/drive/My Drive/Data/2015_Financial_Data.csv', header=0)
data_2016 = pd.read_csv('/content/drive/My Drive/Data/2016_Financial_Data.csv', header=0)
data_2017 = pd.read_csv('/content/drive/My Drive/Data/2017_Financial_Data.csv', header=0)
data_2018 = pd.read_csv('/content/drive/My Drive/Data/2018_Financial_Data.csv', header=0)
#print(data_2014.shape, data_2015.shape, data_2016.shape, data_2017.shape)
#filename='/content/drive/My Drive/Data/2018_Financial_Data.csv'
#print(filename)

#data=pd.read_csv(filename, header=0)
#data=pd.concat([data_2014,data_2015,data_2016,data_2017])

#data = pd.DataFrame(data)
#data_2018 =pd.DataFrame(data_2018)
#data.shape, data_2018.shape


**Step 2 Create a function to identify only those variable as from the dataset that are relevant to target

In [4]:
def transform(t):
  t.rename(columns={'Unnamed: 0':'Company Name'},inplace=True)
  t.replace(np.nan, 0)
  t.drop(t[t['Total assets'] ==0].index, inplace = True) 
  t.drop(t[t['Revenue'] ==0].index, inplace = True) 
  t['Total Asset Turnover Ratio'] = t['Revenue'] / t['Total assets']
  t['Gross Margin Ratio'] = t['Gross Margin'] / t['Revenue']
  t['Net Profit Ratio'] = t['Net Profit Margin']/t['Revenue']
  t['Return on Total Assets Ratio'] = t['Net Income']/t['Total assets']
  features = ['Company Name', 'currentRatio','quickRatio', 'Free Cash Flow margin','debtRatio', 'debtEquityRatio',
              'cashFlowToDebtRatio','Return on Tangible Assets','Total Asset Turnover Ratio',
              'Return on Total Assets Ratio','ROE','Gross Margin Ratio',
              'Net Profit Ratio', 'EPS','EPS Diluted','PE ratio', 'PB ratio','priceEarningsToGrowthRatio', 'Sector','Class']
  t = t[features]

  return t

**Applying the transformation to wrangle the raw dataset**

In [5]:
data2014 = transform(data_2014)
#data2014['2015StockPriceVar'] = data_2014['2015 PRICE VAR [%]']
data2015 = transform(data_2015)
#data2015['2016StockPriceVar'] = data_2015['2016 PRICE VAR [%]']
data2016 = transform(data_2016)
#data_2016['2017StockPriceVar'] = data_2016['2017 PRICE VAR [%]']
data2017 = transform(data_2017)
#data2017['2018StockPriceVar'] = data_2017['2018 PRICE VAR [%]']
data2018 = transform(data_2018)
#data2018['2019StockPriceVar'] = data_2018['2019 PRICE VAR [%]']


Concatenate the four years data between 2014 until 2017 and separately create a data frame to load 2018 data for testing set

In [6]:
data = pd.concat([data2014,data2015,data2016,data2017])
data = pd.DataFrame(data)
data_2018 =pd.DataFrame(data2018)
data.shape, data_2018.shape

((16921, 20), (4194, 20))

In [7]:
# Save the domain based raw data set

## save to xlsx file

filepath = '/content/drive/My Drive/Data/Result/domain_based.xlsx'

data.to_excel(filepath, index=False)

#domain-based= np.sav('/content/drive/My Drive/Data/Result/predictedRFbase.txt', y_pred, fmt='%01d')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4959
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Company Name                  16921 non-null  object 
 1   currentRatio                  14156 non-null  float64
 2   quickRatio                    14159 non-null  float64
 3   Free Cash Flow margin         14931 non-null  float64
 4   debtRatio                     14908 non-null  float64
 5   debtEquityRatio               14908 non-null  float64
 6   cashFlowToDebtRatio           11995 non-null  float64
 7   Return on Tangible Assets     11558 non-null  float64
 8   Total Asset Turnover Ratio    15478 non-null  float64
 9   Return on Total Assets Ratio  15496 non-null  float64
 10  ROE                           14887 non-null  float64
 11  Gross Margin Ratio            15799 non-null  float64
 12  Net Profit Ratio              15411 non-null  float64
 13  EP

**Creating function for Evaluation using Accuracy, Precision**

In [7]:
print(data.iloc[:, -1].unique().size)

if data.iloc[:, -1].unique().size==2:
    def evaluate(y_test, y_pred, y_scores):
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        recall = recall_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        f1 = f1_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        auc = roc_auc_score(y_test, y_scores)
        return [accuracy, precision, recall, f1, auc]
else:

    def evaluate(y_test, y_pred, y_scores):
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
        recall = recall_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
        f1 = f1_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
				#metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))		 

        auc = roc_auc_score(y_test, y_scores)
        return [accuracy, precision, recall, f1, auc]

2


Imputing zeros where there are NaNs. and dropping all price variance columns from the dataset

In [8]:
data = data.drop(['2015 PRICE VAR [%]','2016 PRICE VAR [%]','2017 PRICE VAR [%]','2018 PRICE VAR [%]'], axis=1)
data_2018 = data_2018.drop(['2019StockPriceVar'], axis=1)
data.fillna(0, inplace=True)
data_2018.fillna(0, inplace=True)



KeyError: ignored

In [9]:
data.info()
data_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4959
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Company Name                  16921 non-null  object 
 1   currentRatio                  14156 non-null  float64
 2   quickRatio                    14159 non-null  float64
 3   Free Cash Flow margin         14931 non-null  float64
 4   debtRatio                     14908 non-null  float64
 5   debtEquityRatio               14908 non-null  float64
 6   cashFlowToDebtRatio           11995 non-null  float64
 7   Return on Tangible Assets     11558 non-null  float64
 8   Total Asset Turnover Ratio    15478 non-null  float64
 9   Return on Total Assets Ratio  15496 non-null  float64
 10  ROE                           14887 non-null  float64
 11  Gross Margin Ratio            15799 non-null  float64
 12  Net Profit Ratio              15411 non-null  float64
 13  EP

**Replace all NaN with 0 and blanks with 0 Enable coding to convert Sector**

In [10]:
#data.fillna(0, inplace=True)
#data_2016.fillna(0, inplace=True)

# Categorical boolean mask
categorical_feature_mask = data.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = data.columns[categorical_feature_mask].tolist()
#categorical_cols1 = data_2018.columns[categorical_feature_mask].tolist()

print(categorical_cols)
#Use LabelEncoder() to transfer categorical data to numurical data
le=LabelEncoder()
print(len(categorical_cols))
if len(categorical_cols)!=0:
    data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
    #data_2018[categorical_cols] = data_2018[categorical_cols].apply(lambda col: le.fit_transform(col))
#data[categorical_cols].head(5)
#data_2018[categorical_cols1].head(5)

data=np.array(data)
data_2018=np.array(data_2018)
#If first column is ID, then manuly removed from the dataset





['Company Name', 'Sector']
2


In [ ]:
#identify target baseline
#target_class = 'Class'
#data_target_class = data[target_class].mode()
#data_2018_target_class =data_2018[target_class].mode()
#data_target_class, data_2018_target_class

**Define Train and Test data and Label data**

In [11]:
#X,y=data[:,1:-4], data[:, -1]
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=seed)
#y_train=y_train.astype(int)

X_train,y_train=data[:,1:-3], data[:, -1]
X_test,y_test=data_2018[:,1:-3], data_2018[:, -1]
#ros = RandomOverSampler(random_state=0)

#X_train, y_train = ros.fit_resample(X_train, y_train)


scaler = preprocessing.StandardScaler().fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)



traindata = np.array(X_train2)
trainlabel = np.array(y_train)

testdata = np.array(X_test2)
testlabel = np.array(y_test)

traindata=traindata.astype(float)
trainlabel=trainlabel.astype(int)
testdata=testdata.astype(float)
testlabel = testlabel.astype(int)


**MLP**

In [12]:
def MLP(X_train,y_train,X_test,y_test):
    clf = MLPClassifier(solver='lbfgs', activation= 'relu', alpha=1e-5,hidden_layer_sizes=(10, 5), random_state=seed)
    
    l=X_train[0].size
    lr = np.logspace(-5,-1, 5)
    #print(lr)
    #print(X_train[0].size)
    hz=[(10, 5),(10,5,3)]
    param_grid = {'alpha': lr,'hidden_layer_sizes':hz}

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
					 

    gridcv = gridcv.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]

    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedMLP.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

# Logistic regression

In [13]:
def LogisticRegression(X_train,y_train,X_test,y_test):
    clf = sklearn.linear_model.LogisticRegression(random_state=seed, solver='lbfgs')
    clf = clf.fit(X_train,y_train)
								   
					 
							 
    y_pred = clf.predict(X_test)
    y_scores = clf.predict_proba(X_test)[:,1]
    np.savetxt('/content/drive/My Drive/Data/Result/predictedLR.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

**Running the Decision tree model**

In [14]:
def CART(X_train,y_train,X_test,y_test):
   # clf = tree.DecisionTreeClassifier(max_depth=5)
   # clf = tree.DecisionTreeClassifier(criterion='entropy',max_depth=5,min_samples_leaf=5)

    clf = tree.DecisionTreeClassifier( random_state=seed)

    md=np.arange(1,50,10)
    param_grid = {'max_depth': md}
					 
							 

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    #clf = clf.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    np.savetxt('/content/drive/My Drive/Data/Result/predictedDTbase.txt', y_pred, fmt='%01d')
    print("best parameters:", gridcv.best_params_)
    #tree.plot_tree(clf.fit(X_train,y_train))
    #tree.export_graphviz()
    #dot_data = tree.export_graphviz(clf, out_file=None)
    #graph = graphviz.Source(dot_data)
    #graph.render("tree")
    return evaluate(y_test,y_pred,y_scores)

# Random Forest model*

In [15]:
#randomforest

def RandomForest(X_train,y_train,X_test,y_test):
    clf = ensemble.RandomForestClassifier(max_features=None,random_state=seed)

    md = np.arange(1, 50, 10)
    ne = np.arange(1, 50, 10)
    param_grid = {'max_depth': md,'n_estimators':ne}
					 
							 

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=2)
    #gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    gridcv = gridcv.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedRFbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

**SVM Liner model**

In [16]:
def svm_one_class(X_train,y_train,X_test,y_test):
    _train = preprocessing.normalize(X_train, norm='l2')
    _test = preprocessing.normalize(X_test, norm='l2')
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    LinearSVC(C=1000, class_weight=None, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=seed, tol=0.0001,
     verbose=0)
    y_pred = lin_clf.predict(X_test)
    y_scores = lin_clf.decision_function(X_test)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedSVMbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [17]:
def SVM(X_train,y_train,X_test,y_test):
	
    svm = sklearn.svm.SVC(kernel='rbf')
    C_grid = np.logspace(0, 3, 4)
    gamma_grid = np.logspace(-2, 1, 4)
    param_grid = {'C': C_grid, 'gamma': gamma_grid}
    gridcv = sklearn.model_selection.GridSearchCV(svm, param_grid, verbose=1, cv=2)
    #gridcv = sklearn.model_selection.GridSearchCV(svm, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.decision_function(X_test)
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedSVMNormbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

Naive Bayes model

In [18]:
def Gaussian_NB(X_train,y_train,X_test,y_test):
    gnb = GaussianNB()
		   
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
							 

								
    y_scores  = gnb.fit(X_train, y_train).predict_proba(X_test)[:,1]

    np.savetxt('/content/drive/My Drive/Data/Result/predictedNBbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

 

In [19]:
def kNN(X_train,y_train,X_test,y_test):
    clf = KNeighborsClassifier()
    #n_neighbors=5
    nn=np.arange(1,100,10)
    param_grid = {'n_neighbors': nn}

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedKNNbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

** XG Boost model 

In [20]:
def AdaBoost(X_train,y_train,X_test,y_test):
   # dt_stump = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1)
    dt_stump = DecisionTreeClassifier()
    dt_stump.fit(X_train, y_train)
    dt_stump_err = 1.0 - dt_stump.score(X_test, y_test)

    clf = ensemble.AdaBoostClassifier(base_estimator=dt_stump,learning_rate=0.1, algorithm='SAMME')
    clf= clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    #y_pred = clf.predict(X_test)
    y_scores = clf.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedABbase.txt', y_pred, fmt='%01d')

    return evaluate(y_test,y_pred,y_scores)

In [21]:
def GBDT(X_train,y_train,X_test,y_test):
    n_estimators = [10,50,100]
    learning_rate = [0.01,0.1,1]
    max_depth=[1,5,10]
    param_grid = {'n_estimators': n_estimators, 'learning_rate': learning_rate,'max_depth':max_depth}

    clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=seed)
    #clf=clf.fit(X_train, y_train)

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedBGDTbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

Print the model results for NB

In [22]:
print("NB training")
a1= Gaussian_NB(traindata,trainlabel,testdata,testlabel)
print(a1)
#print("XG Boost training")
#a2=xgboost(traindata,trainlabel,testdata,testlabel)
#print(a2)




NB training


ValueError: ignored

In [23]:
print("LR training")
a2=LogisticRegression(traindata,trainlabel,testdata,testlabel)
print(a2)

LR training


ValueError: ignored

In [ ]:
print("MLP training")
a3=MLP(traindata,trainlabel,testdata,testlabel)
print(a3)



MLP training
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: Convergenc

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

best parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (10, 5)}
[0.6090619307832422, 0.6440718999971404, 0.6090619307832422, 0.6211225741667199, 0.6261105105568017]


In [ ]:
print("RF training")
a5=RandomForest(traindata,trainlabel,testdata,testlabel)
print(a5)

RF training
Fitting 2 folds for each of 25 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 18.4min finished


Fitting 2 folds for each of 25 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 18.3min finished


best parameters: {'max_depth': 41, 'n_estimators': 41}
[0.61316029143898, 0.6717874078198804, 0.61316029143898, 0.6279215737407491, 0.656932849355938]


In [ ]:
print("DT training")
a6=CART(traindata,trainlabel,testdata,testlabel)
print(a6)


DT training
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   35.8s finished


best parameters: {'max_depth': 11}
[0.6038251366120219, 0.6563383014534346, 0.6038251366120219, 0.6184954958970235, 0.5933747179210501]


In [ ]:
print("SVM training")
a4= SVM(traindata,trainlabel,testdata,testlabel)
print(a4)



SVM training
Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed: 43.7min finished


best parameters: {'C': 10.0, 'gamma': 1.0}
[0.6063296903460837, 0.6069006245690755, 0.6063296903460837, 0.6066137595752733, 0.5581443619820503]


In [ ]:

Result = ['a1','a2', 'a3','a4','a5','a6']

In [ ]:
print("Naive_bayes")
print("[accuracy,precision,recall,f1,auc]")
print('a1')

print("NB training")
a2=Gaussian_NB(traindata,trainlabel,testdata,testlabel)
print(a2)

print("LogisticRegression")
print("[accuracy,precision,recall,f1,auc]")
print(a3)

print("SVM")
print("[accuracy,precision,recall,f1,auc]")
print(a4)


print("MLP")
print("[accuracy,precision,recall,f1,auc]")
print(a5)

print("RandomForest")
print("[accuracy,precision,recall,f1,auc]")
print(a6)

print("CART")
print("[accuracy,precision,recall,f1,auc]")
print(a7)


Naive_bayes
[accuracy,precision,recall,f1,auc]
a1
NB training
[0.6816939890710383, 0.581596705337981, 0.6816939890710383, 0.5805745574922684, 0.5026395662740407]
LogisticRegression
[accuracy,precision,recall,f1,auc]


NameError: ignored

In [ ]:
def tsneShow(X,y,Xt,yt):
    yys = np.array(y)
    yyt = np.array(yt)
    for i in range(0, yyt.size):
        if yyt[i] == 0:
            yyt[i] = 2
        elif yyt[i] == 1:
            yyt[i] = 3

    H = np.vstack((X, Xt))
    Y = np.concatenate((yys, yyt), axis=0)

    pca = PCA().fit_transform(H)
    tsne = TSNE(n_components=2, learning_rate=100).fit_transform(H)

    markers = ('.')
    plt.figure(figsize=(12, 6))
    # plt.subplot(121)
    plt.scatter(tsne[:, 0], tsne[:, 1], c=Y, marker=markers[0], cmap=plt.cm.gist_rainbow)

    plt.colorbar()
    plt.show()